# Image classification for Industrial Production

In the real life of Manufacuturing industries, The routine job for the quality control process is inspection. Which is spend a lot of tiny time to interupt manufacturing process. 
In this practice is Image classification in manufacturing. Which is capturing the images in the final process for classifie OK/NG parts

## Download Dataset
The [casting product image data for quality inspection](https://www.kaggle.com/ravirajsinh45/real-life-industrial-dataset-of-casting-product) form Kaggle which is contributed by **Ravirajsinh Dabhi** 

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"intraraksa","key":"72a56f22ab3b3c646e56763a5d6b32d2"}'}

In [3]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download ravirajsinh45/real-life-industrial-dataset-of-casting-product

 83% 83.0M/100M [00:00<00:00, 130MB/s]
100% 100M/100M [00:00<00:00, 137MB/s] 


In [6]:
# !unzip '/content/real-life-industrial-dataset-of-casting-product.zip'

In [1]:
import tensorflow as tf
# from tensorflow.keras.applications.efficientnet import EfficientNetB5
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
from tensorflow.keras.models import Model,Sequential,load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from tensorflow.keras.optimizers import RMSprop,SGD
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import os
import numpy as np
from tensorflow.keras.models import load_model
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# model = Sequential()
# model.add(Conv2D(128,(2,2),padding='valid',input_shape=(300,300,3),activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(128,(2,2),padding='valid',activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(64,(2,2),padding='valid',activation='relu'))
# model.add(Dropout(0.2))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Flatten())
# # model.add(Dense(50,activation='sigmoid'))
# # model.add(Dropout(0.2))
# model.add(Dense(1,activation='sigmoid'))

In [2]:
vgg = VGG16(include_top=False,input_shape=(224,224,3))
for layer in vgg.layers:
  layer.trainable = False

flat = Flatten()(vgg.layers[-1].output)
dense = Dense(128,activation='relu')(flat)
model_output = Dense(1,activation='sigmoid')(dense)
model = Model(vgg.input,model_output)

In [3]:
# opt = SGD()
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
file_path = 'chkpoint/checkpoint.ckpt'
earlystop = EarlyStopping(monitor='val_accuracy',patience=5,mode='max',restore_best_weights=True)
model_check = ModelCheckpoint(file_path,monitor='val_accuracy',save_weights_only=True,save_best_only=True,verbose=1)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
train_path = os.path.join('casting_data/casting_data','train')
test_path = os.path.join('casting_data/casting_data','test')
for subset in os.listdir(train_path):
  print(f'this is folder is {subset} and there are {len(os.listdir(os.path.join(train_path,subset)))} sample')
print('..................')
for subset in os.listdir(test_path):
  print(f'this is folder is {subset} and there are {len(os.listdir(os.path.join(test_path,subset)))} sample')

this is folder is ok_front and there are 2875 sample
this is folder is def_front and there are 3758 sample
..................
this is folder is ok_front and there are 262 sample
this is folder is def_front and there are 453 sample


In [6]:
# fig,ax = plt.subplot(3,3)
os.listdir(train_path)

['ok_front', 'def_front']

In [7]:
train_gen = ImageDataGenerator(height_shift_range=0.1,brightness_range=[1.5,2.0],horizontal_flip=True,vertical_flip = True)
test_gen = ImageDataGenerator()

In [8]:
train_flow = train_gen.flow_from_directory(train_path,target_size=(224,224),class_mode='binary',batch_size=80)
test_flow = train_gen.flow_from_directory(test_path,target_size=(224,224),class_mode='binary',batch_size=32)

Found 6633 images belonging to 2 classes.
Found 715 images belonging to 2 classes.


In [9]:
# history = model.fit_generator(train_flow,epochs=83,steps_per_epoch=len(train_flow),validation_data=test_flow,validation_steps=len(test_flow),callbacks=[model_checkpoint])
history = model.fit_generator(train_flow,epochs=83,validation_data=test_flow,callbacks=[earlystop,model_check])
model.save('defect.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/83
83/83 [==============================] - 191s 2s/step - loss: 2.8780 - accuracy: 0.8394 - val_loss: 0.1123 - val_accuracy: 0.9594

Epoch 00001: val_accuracy improved from -inf to 0.95944, saving model to chkpoint/checkpoint.ckpt
Epoch 2/83
83/83 [==============================] - 120s 1s/step - loss: 0.0686 - accuracy: 0.9804 - val_loss: 0.0528 - val_accuracy: 0.9832

Epoch 00002: val_accuracy improved from 0.95944 to 0.98322, saving model to chkpoint/checkpoint.ckpt
Epoch 3/83
83/83 [==============================] - 120s 1s/step - loss: 0.0463 - accuracy: 0.9876 - val_loss: 0.0509 - val_accuracy: 0.9832

Epoch 00003: val_accuracy did not improve from 0.98322
Epoch 4/83
83/83 [==============================] - 119s 1s/step - loss: 0.0381 - accuracy: 0.9905 - val_loss: 0.0284 - val_accuracy: 0.9944

Epoch 00004: val_accuracy improved from 0.98322 to 0.99441, saving model to chkpoint/checkpoint.ckpt
Epoch 5/83
83/83 [==============================] - 120s 1s/step - loss: 0.03

In [ ]:
6633/80

In [ ]:
7438/128